# Obtain data of QSO from SIMBAD, filtering the objects with data of [Ne II], [Ne III] and [Ne V] in infrared from NED Phot Tables and saving it

In [1]:
# If this notebook will be run in Google Colab, first at all the next code must be run. If the notebook will be run in Jupyter
#then "comment" or omit this part 

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery

In [2]:
from datetime import date
import time
import requests
import numpy as np

import pyvo
from astroquery.ipac.ned import Ned
from astroquery.simbad import Simbad

from pathlib import Path
from requests import Request, Session

import astropy.units as u
from astropy.table import hstack, QTable, Table
from astropy.coordinates import SkyCoord

import getphotNED as pN

In [3]:
path_raw='../Datos/Raw/'
Path(path_raw).mkdir(parents=True, exist_ok=True)
path_phot_ned = '../Datos/Work/Phot_Tables_NED/' #In this path wi will save the photometry tables for each object
Path(path_phot_ned).mkdir(parents=True, exist_ok=True)

In [4]:
service = pyvo.dal.TAPService("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

result0 = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,ids.ids
    ,alltypes.otypes
FROM basic
JOIN ids ON oid = ids.oidref
JOIN alltypes ON oid = alltypes.oidref
WHERE basic.otype = 'QSO'
  AND rvz_redshift <= 0.6
""")

In [5]:
simbad_sample0_QSO = result0.to_table()
obj_name_SMB0=simbad_sample0_QSO["main_id"]
obj_ids_SMB0=simbad_sample0_QSO["ids"]
len(simbad_sample0_QSO)

32835

In [6]:
result1 = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,ids.ids
    ,alltypes.otypes
FROM basic
JOIN ids ON oid = ids.oidref
JOIN alltypes ON oid = alltypes.oidref
WHERE basic.otype = 'QSO'
  AND rvz_redshift > 0.6
  AND rvz_redshift <= 0.7
""")

In [7]:
simbad_sample1_QSO = result1.to_table()
obj_name_SMB1=simbad_sample1_QSO["main_id"]
obj_ids_SMB1=simbad_sample1_QSO["ids"]
len(simbad_sample1_QSO)

18874

In [8]:
result2 = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,ids.ids
    ,alltypes.otypes
FROM basic
JOIN ids ON oid = ids.oidref
JOIN alltypes ON oid = alltypes.oidref
WHERE basic.otype = 'QSO'
  AND rvz_redshift > 0.7
  AND rvz_redshift <= 0.8
""")

In [9]:
simbad_sample2_QSO = result2.to_table()
obj_name_SMB2=simbad_sample2_QSO["main_id"]
obj_ids_SMB2=simbad_sample2_QSO["ids"]
len(simbad_sample2_QSO)

24501

In [10]:
result3 = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,ids.ids
    ,alltypes.otypes
FROM basic
JOIN ids ON oid = ids.oidref
JOIN alltypes ON oid = alltypes.oidref
WHERE basic.otype = 'QSO'
  AND rvz_redshift > 0.8
  AND rvz_redshift <= 0.9
""")

In [11]:
simbad_sample3_QSO = result3.to_table()
obj_name_SMB3=simbad_sample3_QSO["main_id"]
obj_ids_SMB3=simbad_sample3_QSO["ids"]
len(simbad_sample3_QSO)

30254

In [12]:
today = date.today()
today.strftime('%b_%d_%Y')

'Sep_19_2022'

In [14]:
INIT_URL = 'http://vizier.u-strasbg.fr/viz-bin/sed?-c='
urls0 = [INIT_URL+str(simbad_sample0_QSO['ra'][row])+'%20'+
        str(simbad_sample0_QSO['dec'][row])+'&-c.rs=5' for row in range(len(simbad_sample0_QSO))]
simbad_sample0_QSO['cds_url'] = urls0

urls1 = [INIT_URL+str(simbad_sample1_QSO['ra'][row])+'%20'+
        str(simbad_sample1_QSO['dec'][row])+'&-c.rs=5' for row in range(len(simbad_sample1_QSO))]
simbad_sample1_QSO['cds_url'] = urls1

urls2 = [INIT_URL+str(simbad_sample2_QSO['ra'][row])+'%20'+
        str(simbad_sample2_QSO['dec'][row])+'&-c.rs=5' for row in range(len(simbad_sample2_QSO))]
simbad_sample2_QSO['cds_url'] = urls2

In [15]:
simbad_sample0_QSO.write(path_raw+'SMB_'+today.strftime('%b_%d_%Y')+'_QSO_Samples0'+'.vot',
                    format='votable',overwrite=True)
simbad_sample1_QSO.write(path_raw+'SMB_'+today.strftime('%b_%d_%Y')+'_QSO_Samples1'+'.vot',
                    format='votable',overwrite=True)
simbad_sample2_QSO.write(path_raw+'SMB_'+today.strftime('%b_%d_%Y')+'_QSO_Samples2'+'.vot',
                    format='votable',overwrite=True)

cat_smb_sample0 = SkyCoord(ra=simbad_sample0_QSO['ra'], dec=simbad_sample0_QSO['dec'])
cat_smb_sample1 = SkyCoord(ra=simbad_sample1_QSO['ra'], dec=simbad_sample1_QSO['dec'])
cat_smb_sample2 = SkyCoord(ra=simbad_sample2_QSO['ra'], dec=simbad_sample2_QSO['dec'])

Ne_IR_Fq = np.array([19200000000000.0,19300000000000.0,20900000000000.0,21000000000000.0,23400000000000.0])*u.Hz

In [16]:
def FilterbyNeFreq(NEDTable, Freq):
    
    '''
    This function is for search in a photometry
    table is there is information of [Ne] lines
    based to a specific frequency values and 
    return the which of those frequencies was founded'''
    
    NEDTable.remove_rows(np.where(NEDTable['Flux Density'].mask)[0])
    NEDTable.remove_rows(np.where(NEDTable['NED Units'] == 'Jy')[0])
    #NEDTable.remove_rows(np.where(NEDTable['NED Uncertainty'] == '')[0])
    #NEDTable.remove_rows(np.where(NEDTable['NED Uncertainty'] == '+/-...')[0])
    NEDTable.remove_rows(np.where(NEDTable['Observed Passband'] == '[Cl II] line (IRS)')[0])
    NEDTable.remove_rows(np.where(NEDTable['Observed Passband'] == '[Cl II] 14.4 (IRS)')[0])
    NEDTable.remove_rows(np.where(NEDTable['Observed Passband'] == '[Cl II] (IRS)')[0])
    NEDTable.remove_rows(np.where(NEDTable['Observed Passband'] == '[Cl II] 14.37 (IRS)')[0])
    
    f = np.array([NEDTable['Frequency']])*u.Hz #We get the Frequency column from the table
    intersec = np.intersect1d(Freq, f) #Intersection between Fr and Ne_IR_Fq
    #print(intersec)
   
    return intersec

## QSO Sample 0

In [17]:
NeIR_obj_t = QTable(names=('Main_Id','Id_used_NED','RA','DEC','otype_txt','redshift',
                           'rvz_type','ids','otypes','cds_url','SMB_sample_index'),
                    dtype=('O','O','f8','f8','O','f8','U1','O','O','U93','f8'))

Ne_Inf = []

print(f"{'#':4} {'Idx':6} {'Name ID':^27} {'ID used in NED':^27} {'Ne IR Info':^20}")
i=-1
start = time.time()
for ind, id_smb in enumerate(obj_name_SMB0):
    try:       
        #print('\nIdx SIMBAD: ',ind,' main_id: ',id_smb)
        phot_T,name_NED = pN.photNED(id_smb,obj_ids_SMB0[ind],cat_smb_sample0[ind])
        #Save table again for work with it before save the original one
        phot_t,name_ned = pN.photNED(id_smb,obj_ids_SMB0[ind],cat_smb_sample0[ind])
        ins = FilterbyNeFreq(phot_t, Ne_IR_Fq) #Search for the [Ne] frequencies in NED table
        #print(name_ned)
        
        if len(ins)>0: #First filter: If the phot table has infomation of frequencies of Fq            
            #print(ins)
            i=i+1
            phot_T.write(path_phot_ned+id_smb+'_NED_phot_tables'+'.vot', #Save the phot table in a vot table file
                            format='votable',overwrite=True)
            NeIR_obj_row = [id_smb,name_ned,simbad_sample0_QSO["ra"][ind],simbad_sample0_QSO["dec"][ind],
                            simbad_sample0_QSO["otype_txt"][ind],simbad_sample0_QSO["rvz_redshift"][ind],
                            simbad_sample0_QSO["rvz_type"][ind],simbad_sample0_QSO["ids"][ind],
                            simbad_sample0_QSO["otypes"][ind],simbad_sample0_QSO["cds_url"][ind],ind] 
            NeIR_obj_t.add_row(vals=NeIR_obj_row)
            
            if (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins)and (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins)\
            and Ne_IR_Fq[4] in ins: #Second filter to classify according the Ne info after remove empty flux values
                Ne_info = 'NeII|NeIII|NeV'
            elif (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins) and Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII|NeIII'
            elif (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins) and Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII|NeV'
            elif (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins) and (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins):
                Ne_info = 'NeIII|NeV'
            elif Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins:
                Ne_info = 'NeIII'
            elif Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins:
                Ne_info = 'NeV'
            elif Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII'
            else:
                Ne_info = 'No Flux Values' 
            Ne_Inf.append(Ne_info)
            print(f"{'%g'%i:4} {'%g'%ind:6} {'%s'%id_smb:^27} {'%s'%name_ned:^27} {'%s'%Ne_info:^20}")
    
    except KeyError:
        pass

end = time.time()

print('\nTotal objects with lines emission of NeII or NeIII or NeV in IR  found: ',len(NeIR_obj_t))
print('Execution Time(seg): ' + str(end - start))
print('Execution Time(min): ' + str((end - start)/60))
print('Execution Time(hrs): ' + str((end - start)/3600))

#    Idx              Name ID                 ID used in NED             Ne IR Info     
0    2224     2MASS J01572105+1712484     2MASS J01572104+1712481           NeII        
1    4785     2MASS J00505570+2933281     2MASS J00505569+2933280           NeV         
2    6532     2MASS J17144277+2602485    WISEA J171442.76+260248.5          NeII        
3    7042          QSO B1549-790                1Jy 1549-79            NeII|NeIII|NeV   
4    11382    2MASS J17193331+5927430     2MASS J17193331+5927430          NeIII        
5    11727           3C 263.1                    3C 263.1                NeII|NeIII     
6    12467        QSO B0125-0635              GSC 04684-01057            NeII|NeIII     
7    14121    2MASS J23444957+1221434     2MASX J23444953+1221430        NeIII|NeV      
8    14254    2MASS J09215939+4509126     2MASS J09215939+4509126          NeIII        
9    14869   [VV2000] J201329.8-414735   WISEA J201329.54-414734.8       NeII|NeIII     
10   15037    2MASS J

## QSO Sample 1

In [18]:
print(f"{'#':4} {'Idx':6} {'Name ID':^27} {'ID used in NED':^27} {'Ne IR Info':^20}")
i=-1
start = time.time()
for ind, id_smb in enumerate(obj_name_SMB1):
    try:       
        #print('\nIdx SIMBAD: ',ind,' main_id: ',id_smb)
        phot_T,name_NED = pN.photNED(id_smb,obj_ids_SMB1[ind],cat_smb_sample1[ind])
        #Save table again for work with it before save the original one
        phot_t,name_ned = pN.photNED(id_smb,obj_ids_SMB1[ind],cat_smb_sample1[ind])
        ins = FilterbyNeFreq(phot_t, Ne_IR_Fq) #Search for the [Ne] frequencies in NED table
        #print(name_ned)
        
        if len(ins)>0: #First filter: If the phot table has infomation of frequencies of Fq            
            #print(ins)
            i=i+1
            phot_T.write(path_phot_ned+id_smb+'_NED_phot_tables'+'.vot', #Save the phot table in a vot table file
                            format='votable',overwrite=True)
            NeIR_obj_row = [id_smb,name_ned,simbad_sample1_QSO["ra"][ind],simbad_sample1_QSO["dec"][ind],
                            simbad_sample1_QSO["otype_txt"][ind],simbad_sample1_QSO["rvz_redshift"][ind],
                            simbad_sample1_QSO["rvz_type"][ind],simbad_sample1_QSO["ids"][ind],
                            simbad_sample1_QSO["otypes"][ind],simbad_sample1_QSO["cds_url"][ind],ind] 
            NeIR_obj_t.add_row(vals=NeIR_obj_row)
            
            if (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins)and (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins)\
            and Ne_IR_Fq[4] in ins: #Second filter to classify according the Ne info after remove empty flux values
                Ne_info = 'NeII|NeIII|NeV'
            elif (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins) and Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII|NeIII'
            elif (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins) and Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII|NeV'
            elif (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins) and (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins):
                Ne_info = 'NeIII|NeV'
            elif Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins:
                Ne_info = 'NeIII'
            elif Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins:
                Ne_info = 'NeV'
            elif Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII'
            else:
                Ne_info = 'No Flux Values' 
            Ne_Inf.append(Ne_info)
            print(f"{'%g'%i:4} {'%g'%ind:6} {'%s'%id_smb:^27} {'%s'%name_ned:^27} {'%s'%Ne_info:^20}")
    
    except KeyError:
        pass

end = time.time()

print('\nTotal objects with lines emission of NeII or NeIII or NeV in IR  found: ',len(NeIR_obj_t))
print('Execution Time(seg): ' + str(end - start))
print('Execution Time(min): ' + str((end - start)/60))
print('Execution Time(hrs): ' + str((end - start)/3600))

#    Idx              Name ID                 ID used in NED             Ne IR Info     

Total objects with lines emission of NeII or NeIII or NeV in IR  found:  17
Execution Time(seg): 30983.246644735336
Execution Time(min): 516.3874440789223
Execution Time(hrs): 8.606457401315371


## QSO Sample 2

In [19]:
print(f"{'#':4} {'Idx':6} {'Name ID':^27} {'ID used in NED':^27} {'Ne IR Info':^20}")
i=-1
start = time.time()
for ind, id_smb in enumerate(obj_name_SMB2):
    try:       
        #print('\nIdx SIMBAD: ',ind,' main_id: ',id_smb)
        phot_T,name_NED = pN.photNED(id_smb,obj_ids_SMB2[ind],cat_smb_sample2[ind])
        #Save table again for work with it before save the original one
        phot_t,name_ned = pN.photNED(id_smb,obj_ids_SMB2[ind],cat_smb_sample2[ind])
        ins = FilterbyNeFreq(phot_t, Ne_IR_Fq) #Search for the [Ne] frequencies in NED table
        #print(name_ned)
        
        if len(ins)>0: #First filter: If the phot table has infomation of frequencies of Fq            
            #print(ins)
            i=i+1
            phot_T.write(path_phot_ned+id_smb+'_NED_phot_tables'+'.vot', #Save the phot table in a vot table file
                            format='votable',overwrite=True)
            NeIR_obj_row = [id_smb,name_ned,simbad_sample2_QSO["ra"][ind],simbad_sample2_QSO["dec"][ind],
                            simbad_sample2_QSO["otype_txt"][ind],simbad_sample2_QSO["rvz_redshift"][ind],
                            simbad_sample2_QSO["rvz_type"][ind],simbad_sample2_QSO["ids"][ind],
                            simbad_sample2_QSO["otypes"][ind],simbad_sample2_QSO["cds_url"][ind],ind] 
            NeIR_obj_t.add_row(vals=NeIR_obj_row)
            
            if (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins)and (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins)\
            and Ne_IR_Fq[4] in ins: #Second filter to classify according the Ne info after remove empty flux values
                Ne_info = 'NeII|NeIII|NeV'
            elif (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins) and Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII|NeIII'
            elif (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins) and Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII|NeV'
            elif (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins) and (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins):
                Ne_info = 'NeIII|NeV'
            elif Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins:
                Ne_info = 'NeIII'
            elif Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins:
                Ne_info = 'NeV'
            elif Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII'
            else:
                Ne_info = 'No Flux Values' 
            Ne_Inf.append(Ne_info)
            print(f"{'%g'%i:4} {'%g'%ind:6} {'%s'%id_smb:^27} {'%s'%name_ned:^27} {'%s'%Ne_info:^20}")
    
    except KeyError:
        pass

end = time.time()

print('\nTotal objects with lines emission of NeII or NeIII or NeV in IR  found: ',len(NeIR_obj_t))
print('Execution Time(seg): ' + str(end - start))
print('Execution Time(min): ' + str((end - start)/60))
print('Execution Time(hrs): ' + str((end - start)/3600))

#    Idx              Name ID                 ID used in NED             Ne IR Info     
0    10514            3C 254                      3C 254                   NeIII        

Total objects with lines emission of NeII or NeIII or NeV in IR  found:  18
Execution Time(seg): 24631.354742527008
Execution Time(min): 410.5225790421168
Execution Time(hrs): 6.84204298403528


In [ ]:
NeIR_obj_t['NED_Ne_IR_info'] = Ne_Inf #Add a column to save the Ne info
NeIR_obj_t.show_in_notebook()

In [ ]:
NeIR_obj_t.write(path_raw+'Obj_QSO_Ne-IR_'+today.strftime('%b_%d_%Y')+'.vot',
                    format='votable',overwrite=True)

In [ ]:
NeII_NeIII_NeV_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeII|NeIII|NeV')[0] #List of objects with NeII, NeIII and NeV data
NeII_NeIII_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeII|NeIII')[0] #List of objects with NeII and NeIII data
NeII_NeV_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeII|NeV')[0] #List of objects with NeII and NeV data
NeIII_NeV_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeIII|NeV')[0] #List of objects with NeIII and NeV data
NeIII_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeIII')[0] #List of objects with NeIII data
NeV_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeV')[0] #List of objects with NeV data
NeII_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeII')[0] #List of objects with NeII data
No_Ne_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'No Flux Values')[0] #List of objects with NeII data
print('Total objects found with NeII, NeIII and NeV emission lines: ',len(NeII_NeIII_NeV_IR))
print('Total objects found with NeII and NeIII emission lines: ',len(NeII_NeIII_IR))
print('Total objects found with NeII and NeV emission lines: ',len(NeII_NeV_IR))
print('Total objects found with NeIII and NeV emission lines: ',len(NeIII_NeV_IR))
print('Total objects found with NeIII emission lines: ',len(NeIII_IR))
print('Total objects found with NeV emission lines: ',len(NeV_IR))
print('Total objects found with NeII emission lines: ',len(NeII_IR))
print('Total objects found with No Ne flux values: ',len(No_Ne_IR))

In [ ]:
%reload_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

In [ ]:
idS = ['3C 244.1'] #Id to search
idx = [np.where(simbad_sample0_QSO['main_id'] == idt)[0] for idt in idS]
print(obj_name_SMB0[idx[0]])
print('SMB sample index: ',idx[0])